In [ ]:
import logging
import matplotlib.pyplot as plt
import numpy as np

# Import the main class
from main import MainOptimizer
logger = logging.getLogger()
logger.setLevel(logging.INFO)

print("Setup complete. Ready to initialize the optimizer.")


In [ ]:
try:
    optimizer = MainOptimizer()
    
    # Start the optimization process
    optimizer.run()
    
except SystemExit as e:
    print(f"Optimizer stopped with a critical configuration error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    # Use the logging traceback for more detail
    logging.exception("Detailed traceback:")

print("\n--- Optimization Finished ---")

### The following script parses the ga_checkpoint.json file to find the optimal fuel enrichment pattern from the completed optimization run. It identifies the best solution by locating the cycle with the highest stored fitness score and then extracts that cycle's corresponding enrichment values, k-effective, and power peaking factor.

In [ ]:
import json
import numpy as np

try:
    # Update the path to where your new checkpoint file is saved
    with open('data/ga_checkpoint.json', 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    print("Error: 'data/ga_checkpoint.json' not found. Please ensure the path is correct.")
    exit()
except json.JSONDecodeError:
    print("Error: Could not decode JSON. The checkpoint file might be corrupted or empty.")
    exit()


# --- Extract Data from the 'history' List ---
history = data.get('history')
if not history:
    print("Error: 'history' key not found or is empty in the checkpoint file.")
    exit()

# --- Find the best cycle based on the pre-calculated, stored fitness score ---
best_fitness_from_history = -np.inf
best_cycle_data = None

for cycle_record in history:
    # Directly get the pre-calculated fitness from the record
    current_fitness = cycle_record.get('fitness')

    # Skip any records that might be missing a fitness value
    if current_fitness is None:
        continue

    # If this is the best fitness found so far, store the entire record
    if current_fitness > best_fitness_from_history:
        best_fitness_from_history = current_fitness
        best_cycle_data = cycle_record

# --- Display the results ---
if best_cycle_data:
    # Extract details from the best cycle found
    best_cycle_num = best_cycle_data.get('cycle')
    best_keff = best_cycle_data.get('keff')
    best_ppf = best_cycle_data.get('ppf')
    best_individual = best_cycle_data.get('individual')
    # Use the fitness score directly from the record
    actual_best_fitness = best_cycle_data.get('fitness')

    average_enrichment = None
    if best_individual:
        average_enrichment = np.mean(best_individual)

    print("--- Analysis of ga_checkpoint.json (Using Stored Fitness) ---")
    print(f"The cycle with the highest stored fitness is: {best_cycle_num}")
    print(f"The highest fitness found in history is: {actual_best_fitness:.6f}")
    print(f"The corresponding Keff for this cycle is: {best_keff:.6f}")
    print(f"The corresponding PPF for this cycle is: {best_ppf:.6f}")
    print(f"The corresponding enrichment values are: {best_individual}")
    if average_enrichment is not None:
        print(f"The average enrichment for this individual is: {average_enrichment:.6f}")
else:
    print("Could not determine the best individual. The history might be empty or missing fitness data.")


### The following script loads historical data from ga_checkpoint file, extracting the fitness scores and the percentage errors for both PPF and k-effective from each cycle. It then uses matplotlib to generate and save three separate plots that visualize the trends of these key performance metrics over the entire optimization run.

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

try:
    # Update the path to where your new checkpoint file is saved
    with open('data/ga_checkpoint.json', 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    print("Error: 'data/ga_checkpoint.json' not found. Please ensure the path is correct.")
    exit()
except json.JSONDecodeError:
    print("Error: Could not decode JSON. The checkpoint file might be corrupted or empty.")
    exit()


# --- Extract Data from the 'history' List ---
history = data.get('history')
if not history:
    print("Error: 'history' key not found or is empty in the checkpoint file.")
    exit()

# Extract the lists, using .get() for safety and handling potential None values
fitness_scores = [record.get('fitness') for record in history if record.get('fitness') is not None]
ppf_errors = [record.get('ppf_error_percent') for record in history if record.get('ppf_error_percent') is not None]
keff_errors = [record.get('keff_error_percent') for record in history if record.get('keff_error_percent') is not None]

if not fitness_scores:
    print("No valid fitness data found in the history.")
    exit()

# --- Matplotlib Styling (Applied to all plots) ---
plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 14,
    'axes.titlesize': 20,
    'axes.labelsize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14,
    'lines.linewidth': 2,
    'lines.markersize': 8
})


# =======================================================
# === 1. Fitness Score vs. OpenMC Cycle Plot          ===
# =======================================================
print("--- Generating Fitness Plot ---")
min_fitness = min(fitness_scores)
max_fitness = max(fitness_scores)
min_index = fitness_scores.index(min_fitness)
max_index = fitness_scores.index(max_fitness)

print(f"\033[94mFitness: Min = {min_fitness:.10f} (Cycle {min_index}), Max = {max_fitness:.10f} (Cycle {max_index})\033[0m")

plt.figure(figsize=(10, 7))
plt.plot(fitness_scores, marker='o', linestyle='-', color='darkred', label='Fitness Score')
plt.scatter(min_index, min_fitness, color='blue', s=100, zorder=5, label=f'Min: {min_fitness:.4f}')
plt.scatter(max_index, max_fitness, color='orange', s=100, zorder=5, label=f'Max: {max_fitness:.4f}')

# Add a text box for annotations
bbox_props = dict(boxstyle='round,pad=0.4', facecolor='lightyellow', edgecolor='black', alpha=0.8)
textstr = (f"Max Fitness: {max_fitness:.6f} (Cycle {max_index})\n"
           f"Min Fitness: {min_fitness:.6f} (Cycle {min_index})")
plt.text(0.95, 0.5, textstr, transform=plt.gca().transAxes, fontsize=13,
         verticalalignment='center', horizontalalignment='right', bbox=bbox_props)

plt.title('Fitness Score vs. OpenMC Cycle', color='darkred')
plt.xlabel('OpenMC Cycle', color='darkblue')
plt.ylabel('Fitness Score', color='darkblue')
plt.xlim(-1, len(fitness_scores))
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best')
plt.tight_layout()
plt.savefig("fitness_score.png", dpi=300)
plt.show()


# =======================================================
# === 2. PPF Error Percent vs. OpenMC Cycle Plot      ===
# =======================================================
if ppf_errors:
    print("\n--- Generating PPF Error Plot ---")
    avg_ppf = np.mean(ppf_errors)
    min_ppf = min(ppf_errors)
    max_ppf = max(ppf_errors)
    min_ppf_idx = ppf_errors.index(min_ppf)
    max_ppf_idx = ppf_errors.index(max_ppf)
    print(f"\033[94mPPF Error: Avg = {avg_ppf:.4f}%, Min = {min_ppf:.4f}% (Cycle {min_ppf_idx}), Max = {max_ppf:.4f}% (Cycle {max_ppf_idx})\033[0m")

    plt.figure(figsize=(10, 7))
    plt.plot(ppf_errors, marker='o', linestyle='-', color='royalblue', label='PPF Error %')
    plt.scatter(min_ppf_idx, min_ppf, color='green', s=100, zorder=5, label=f'Min: {min_ppf:.2f}%')
    plt.scatter(max_ppf_idx, max_ppf, color='darkorange', s=100, zorder=5, label=f'Max: {max_ppf:.2f}%')
    plt.axhline(y=avg_ppf, color='red', linestyle='--', linewidth=1.5, label=f'Avg: {avg_ppf:.2f}%')

    plt.title('PPF Error Percent vs. OpenMC Cycle', color='darkred')
    plt.xlabel('OpenMC Cycle', color='darkblue')
    plt.ylabel('PPF Error Percent (%)', color='darkblue')
    plt.ylim(bottom=0)
    plt.xlim(-1, len(ppf_errors))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig("ppf_error_percent.png", dpi=300)
    plt.show()
else:
    print("No PPF error data to plot.")

# =======================================================
# === 3. Keff Error Percent vs. OpenMC Cycle Plot     ===
# =======================================================
if keff_errors:
    print("\n--- Generating Keff Error Plot ---")
    avg_keff = np.mean(keff_errors)
    min_keff = min(keff_errors)
    max_keff = max(keff_errors)
    min_keff_idx = keff_errors.index(min_keff)
    max_keff_idx = keff_errors.index(max_keff)
    print(f"\033[94mKeff Error: Avg = {avg_keff:.4f}%, Min = {min_keff:.4f}% (Cycle {min_keff_idx}), Max = {max_keff:.4f}% (Cycle {max_keff_idx})\033[0m")

    plt.figure(figsize=(10, 7))
    plt.plot(keff_errors, marker='o', linestyle='-', color='purple', label='Keff Error %')
    plt.scatter(min_keff_idx, min_keff, color='green', s=100, zorder=5, label=f'Min: {min_keff:.4f}%')
    plt.scatter(max_keff_idx, max_keff, color='darkorange', s=100, zorder=5, label=f'Max: {max_keff:.4f}%')
    plt.axhline(y=avg_keff, color='red', linestyle='--', linewidth=1.5, label=f'Avg: {avg_keff:.4f}%')

    plt.title('Keff Error Percent vs. OpenMC Cycle', color='darkred')
    plt.xlabel('OpenMC Cycle', color='darkblue')
    plt.ylabel('Keff Error Percent (%)', color='darkblue')
    plt.xlim(-1, len(keff_errors))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig("keff_error_percent.png", dpi=300)
    plt.show()
else:
    print("No Keff error data to plot.")